# Importando Biblioteca e Dados

In [1]:
import pandas as pd
import numpy as np

In [314]:
candidatos = pd.read_csv('dados/consulta_cand_2020_SP.csv', sep =';', encoding='latin-1')
bens = pd.read_csv('dados/bem_candidato_2020_SP.csv',  sep =';', encoding='latin-1')

# Limpando os Dados

## Dados da tabela de candidatos

In [315]:
# Aqui você deve inserir o nome da cidade (ou cidades) desejada. O nome precisa estar em caixa alta e com acento
# Em caso de mais de uma cidade, basta adicionar "|" (que significa "ou") e adicionar as outras cidades
candidatos_cidade = candidatos.query('NM_UE == "CIDADE1" | `NM_UE` == "CIDADE2"')

In [316]:
# Vamos começar a remover os candidatos que já não participam da disputa e foram considerados Inaptos pelo TSE
# Escolhi retirar os inaptos ao invés de aceitar apenas os aptos pois o TSE pode ainda estar julgando alguma candidatura e não há sentido em eliminá-la do dataset ainda
candidatos_aptos = candidatos_cidade.query('DS_SITUACAO_CANDIDATURA != "INAPTO"')

In [317]:
# Alguns candidatos são cadastrados duas vezes, por algum erro. Esse comando elimina os candidatos com mesmo CPF.
# É melhor se basear pelo CPF do que pelo nome, pois duas pessoas diferentes podem ter o mesmo nome
# Primeiro temos que transformar o CPF em texto, pois o programa entende que ele é um número. Ou seja, ele pode passar por uma operação matemática.
# Como sabemos que o CPF é um número fixo, inalterável, vamos avisar o programa para mante-lo assim
candidatos_aptos['NR_CPF_CANDIDATO'] = candidatos_aptos['NR_CPF_CANDIDATO'].astype(str)
candidatos_aptos.drop_duplicates(subset = 'NR_CPF_CANDIDATO',keep = 'last', inplace = True)

C:\Users\felip\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\felip\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Dados da tabela de patrimônio

In [318]:
# Primeiro é preciso transformar o SQ, o código de identificação do candidato, em string, assim como foi feito com o CPF
bens['SQ_CANDIDATO'] = bens['SQ_CANDIDATO'].astype(str)

In [319]:
# Alguns valores de bens da tabela estão negativos. Para lidar com isso aproveitei que os valores dos bens estão como string e substitui o caractere "-" por nada
bens["VR_BEM_CANDIDATO"] = [x.replace("-", "") for x in bens["VR_BEM_CANDIDATO"]]

In [320]:
# Agora vamos lidar com as entradas repetidas, ou seja, linhas que estão duplicadas. Para isso criei uma nova viarável de conferência
bens['REPETIDO']=bens.SQ_CANDIDATO+bens.DS_BEM_CANDIDATO+bens.VR_BEM_CANDIDATO
# A partir dessa variável é possível tirar as linhas que apresentam a mesma informação sobre: Candidato, Descrição do Bem, Valor do Bem
bens.drop_duplicates(subset = "REPETIDO", keep = 'last', inplace = True)

In [321]:
# Os dados de valor do bem estão em formato string (ou seja, texto), e estão separados por virgula (e o programa usa o separado ".")
# Nesse primeiro passo vamos transformar a vírgula em ponto e em seguida transformar em número, para o programa poder fazer a soma mais a frente
bens["VR_BEM_CANDIDATO"] = [x.replace(",", ".") for x in bens["VR_BEM_CANDIDATO"]]
bens['VR_BEM_CANDIDATO'] = bens['VR_BEM_CANDIDATO'].astype(float)

In [322]:
# Como fizemos com os candidatos, vamos separar somente as cidades desejadas
bens_cidade = bens.query('NM_UE == "CIDADE1" | `NM_UE` == "CIDADE2"')

# Criando uma nova tabela

In [323]:
# Com os dados da tabela de bens limpos, podemos fazer a soma desses bens, para obter o patrimônio total
bens_somados = bens_cidade.groupby('SQ_CANDIDATO').agg({ 'VR_BEM_CANDIDATO' : sum})
bens_somados.reset_index(level=0, inplace=True)

In [324]:
# Criei uma tabela com apenas alguns dados da tabela de candidatos
# Neste caso selecionei: Nome, Nome na Urna, Partido, Cidade, Cargo, Ocupação, Genêro e Raça
candidatos_aptos = candidatos_aptos[['SQ_CANDIDATO', 
                                     'NM_CANDIDATO', 
                                     'NM_URNA_CANDIDATO',
                                     'NM_UE',
                                     'SG_PARTIDO', 
                                     'DS_CARGO', 
                                     'DS_OCUPACAO', 
                                     'DS_GENERO', 
                                     'DS_COR_RACA']]

In [325]:
candidatos_aptos['SQ_CANDIDATO'] = candidatos_aptos['SQ_CANDIDATO'].astype(str)

In [326]:
# Uni as tabelas através do SQ
dataset = candidatos_aptos.set_index('SQ_CANDIDATO').join(bens_somados.set_index('SQ_CANDIDATO'))

In [327]:
# Preenchendo com 0 os valores nulos
dataset['VR_BEM_CANDIDATO'].fillna(0, inplace = True)

In [328]:
# renomeando as colunas
dataset.rename(columns= {'NM_CANDIDATO' : 'CANDIDATO',
                         'NM_URNA_CANDIDATO' : 'NOME_URNA', 
                         'NM_UE' : 'MUNICIPIO',
                         'SG_PARTIDO' : 'PARTIDO', 
                         'DS_CARGO' : 'CARGO', 
                         'DS_OCUPACAO' : 'OCUPACAO',  
                         'DS_GENERO' : 'GENERO', 
                         'DS_COR_RACA' : 'COR',  
                         'VR_BEM_CANDIDATO' : 'VALOR_PATRIMONIO'},
              inplace = True)

In [329]:
dataset.sort_values(by= "VALOR_PATRIMONIO", ascending = False, inplace = True)

In [330]:
dataset.reset_index(inplace = True)

In [331]:
dataset.drop(axis = 'columns', labels = 'SQ_CANDIDATO', inplace=True)

In [334]:
dataset = round(dataset,2)

# Exportando para uma nova tabela

In [338]:
dataset.to_csv('dados/ranking_patrimonio_candidatos.csv', sep =';', encoding='utf-8', index = False)